In [ ]:
import requests
from bs4 import BeautifulSoup


def scrape_match_summary_links():
    url = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament'
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'lxml')

    links = []
    allRows = soup.select('table.ds-w-full.ds-table.ds-table-xs.ds-table-auto.ds-w-full.ds-overflow-scroll.ds-scrollbar-hide')
    
    for row in allRows:
        tds = row.find_all('td')
        a = 7
        while(a<len(tds)):
            rowURL = "https://www.espncricinfo.com" + tds[a+6].find('a')['href']
            links.append(rowURL)
            a = a+7
    return links



In [ ]:

def scrape_batting_summary(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    

    
    #matchInfo = soup.find('span', text="Match Details").find_next('span').text
    #team1 = matchInfo.split(" Vs ")[0]
    #team2 = matchInfo.split(" Vs ")[1]
    

    tables = soup.find_all('table', class_='ci-scorecard-table')
    
    firstInningRows = tables[0].find_all('tr', lambda tag: len(tag.find_all('td')) >= 8)
    secondInningRows = tables[1].find_all('tr', lambda tag: len(tag.find_all('td')) >= 8)

    battingSummary = []

    def extract_batting_data(rows, team):
        for index, row in enumerate(rows):
            tds = row.find_all('td')
            battingSummary.append({
                "teamInnings": team,
                "battingPos": index + 1,
                "batsmanName": tds[0].find('a').find('span').text.strip(),
                "dismissal": tds[1].find('span').text,
                "runs": tds[2].find('strong').text,
                "balls": tds[3].text,
                "4s": tds[5].text,
                "6s": tds[6].text,
                "SR": tds[7].text
            })

    extract_batting_data(firstInningRows)
    extract_batting_data(secondInningRows)

    return battingSummary


In [ ]:

if __name__ == '__main__':
    
    match_summary_links = scrape_match_summary_links()

    
    batting_summaries = []
    for link in match_summary_links:
        print(link)
        batting_summaries.extend(link)
        

reslut = {"links" : match_summary_links}




import json
json_file_path = 'links.json'


with open(json_file_path, 'w') as json_file:
    json.dump(reslut, json_file, indent=4)
    